In [ ]:
# Authors: Daichi Yoshikawa <daichi.yoshikawa@gmail.com>
# License: BSD 3 clause

from __future__ import absolute_import

import sys
sys.path.append('../../')

import pickle
import numpy as np
import dnn
from dnn.neuralnet import NeuralNetwork
from dnn.utils.nn_utils import scale_normalization

from dnn.training.optimizer import Adam, AdaGrad, AdaDelta, Momentum
from dnn.training.random_weight import RandomWeight
from dnn.training.loss_function import LossFunction

from dnn.layers.affine import AffineLayer
from dnn.layers.activation import Activation, ActivationLayer
from dnn.layers.dropout import DropoutLayer
from dnn.layers.batch_norm import BatchNormLayer

def get_mnist():
    x = np.load('input1.npy')
    x = np.r_[x, np.load('input2.npy')]
    x = np.r_[x, np.load('input3.npy')]
    x = np.r_[x, np.load('input4.npy')]
    x = np.r_[x, np.load('input5.npy')]
    x = np.r_[x, np.load('input6.npy')]
    x = np.r_[x, np.load('input7.npy')]
    x = x.astype(float)
    
    y = np.load('output.npy')
    y = y.astype(float)
    
    return x, y

dtype = np.float32
model = NeuralNetwork(input_shape=(784), dtype=dtype)
model.add(DropoutLayer(drop_ratio=0.2))
model.add(AffineLayer(shape=(784, 392), random_weight=RandomWeight.Type.he))
model.add(BatchNormLayer())
model.add(ActivationLayer(activation=Activation.Type.relu))
model.add(DropoutLayer(drop_ratio=0.5))
model.add(AffineLayer(shape=(392, 10), random_weight=RandomWeight.Type.xavier))
model.add(BatchNormLayer())
model.add(ActivationLayer(activation=Activation.Type.softmax))
model.compile()

x, y = get_mnist()
scale_normalization(x)

optimizer = AdaGrad(learning_rate=5e-2, weight_decay=1e-3, dtype=dtype)

lc = model.fit(
        x=x,
        y=y,
        epochs=20,
        batch_size=100,
        optimizer=optimizer,
        loss_function=LossFunction.Type.multinomial_cross_entropy,
        learning_curve=True,
        shuffle=True,
        shuffle_per_epoch=True,
        test_data_ratio=0.142857 # Use 60,000 for training and 10,000 for test.
)

lc.plot(figsize=(8,10), fontsize=12)

100.00% epoch: 1 loss: 0.10171 acc: 0.970 loss(test): 0.12431 acc(test): 0.962
100.00% epoch: 2 loss: 0.07596 acc: 0.977 loss(test): 0.10239 acc(test): 0.968
100.00% epoch: 3 loss: 0.05871 acc: 0.983 loss(test): 0.08760 acc(test): 0.972
100.00% epoch: 4 loss: 0.04979 acc: 0.986 loss(test): 0.08043 acc(test): 0.975
100.00% epoch: 5 loss: 0.04315 acc: 0.987 loss(test): 0.07545 acc(test): 0.978
100.00% epoch: 6 loss: 0.03942 acc: 0.988 loss(test): 0.07244 acc(test): 0.978
100.00% epoch: 7 loss: 0.03334 acc: 0.990 loss(test): 0.06756 acc(test): 0.979
100.00% epoch: 8 loss: 0.03157 acc: 0.990 loss(test): 0.06749 acc(test): 0.979
100.00% epoch: 9 loss: 0.02724 acc: 0.992 loss(test): 0.06461 acc(test): 0.981
100.00% epoch:10 loss: 0.02489 acc: 0.993 loss(test): 0.06202 acc(test): 0.981
100.00% epoch:11 loss: 0.02385 acc: 0.993 loss(test): 0.06250 acc(test): 0.982
100.00% epoch:12 loss: 0.02353 acc: 0.993 loss(test): 0.06373 acc(test): 0.981
100.00% epoch:13 loss: 0.02132 acc: 0.994 loss(test)